In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sys,os
sys.path.append(os.path.realpath('..'))

from core.data import load_records

In [2]:
source = './records/macho/'
batch = load_records(source, batch_size=2)

In [13]:
def create_padding_mask(tensor, lengths):
    ''' Create mask given a tensor and true length '''
    lengths_transposed = tf.expand_dims(lengths, 1)
    rangex = tf.range(0, tf.shape(tensor)[1], 1)
    range_row = tf.expand_dims(rangex, 0)
    # Use the logical operations to create a mask
    mask = tf.greater(range_row, lengths_transposed)
    return tf.cast(mask, tf.float32)

def create_prediction_mask(tensor, frac=0.15):
    time_steps = tensor.shape[1] 
    indices = tf.map_fn(lambda x: tf.random.shuffle(tf.range(time_steps))[:int(time_steps*frac)], 
                        tf.range(tensor.shape[0]))
    one_hot = tf.one_hot(indices, time_steps)
    mask    = tf.reduce_sum(one_hot, 1)
    return tf.cast(mask, tf.float32)

def create_mask(tensor, length=None):
    mask = create_prediction_mask(tensor)
    if length is not None:
        mask_1 = create_padding_mask(tensor, length)
        mask = tf.maximum(mask, mask_1)
        
    return mask

In [100]:
for i, (x1, x2, length, cls) in enumerate(batch):

    mask1 = create_mask(x1)
    mask2 = create_mask(x2)
    cls = tf.tile(cls, [1, 1])
    sep = tf.tile([[102.]], [x1.shape[0],1])
    mask = tf.concat([cls, mask1, sep, mask2], 1)
    dim_mask = tf.shape(mask)[1]
    mask = tf.tile(mask, [1, dim_mask])
    mask = tf.reshape(mask, [tf.shape(mask)[0], dim_mask, dim_mask])
    mask = tf.expand_dims(mask, 1)
    print(mask.shape)
    cls = tf.tile(tf.expand_dims(cls, 2), [1, 1, x1.shape[-1]])
    sep = tf.tile([[[102.]]], [x1.shape[0],1,x1.shape[-1]])
    inputs = tf.concat([cls, x1, sep, x2], 1)
    
    
    break
    plt.figure()

    cls_lab = tf.slice(x, [0, 0], [-1, 1])    
    index = tf.where(x == 101)[0, -1]
    x_inp = tf.slice(x, [0, 1], [-1, index-1])
    x_tar = tf.slice(x, [0, index+1], [-1, x.shape[1]-index-1])
    
    t_inp = tf.slice(t, [0, 1], [-1, index-1])
    t_tar = tf.slice(t, [0, index+1], [-1, t.shape[1]-index-1])

    plt.plot(t_inp[0], x_inp[0], color='darkred', label=l1.numpy()[0].decode("utf-8"))
    plt.plot(1+t_tar[0], x_tar[0], color='darkblue', label=l2.numpy()[0].decode("utf-8")) 


    if cls_lab.numpy()[0] == 0:
        plt.title('No Random Portion')
    else:
        plt.title('Using Random Portion')
    plt.legend()
    
    if i == 5:break
    plt.xlabel('Time Step')
    plt.ylabel('Magnitude')
    plt.grid(True)
    plt.show()


(2, 1, 202, 202)
